# Infrastructure

In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  24125      0 --:--:-- --:--:-- --:--:-- 24172
OK
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: http://packages.cloud.google.com/apt/dists/gcsfuse-bionic/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.4 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../gcsfuse_2.0.1_amd64.deb ...
Unpacking gcsfuse (2.0.1) ...
Setting up gcsfuse (2.0.1) ...


In [56]:
!mkdir gcs
!mkdir efficientnet
!gcsfuse --implicit-dirs --rename-dir-limit=100 --max-conns-per-host=100 "hms_applied_cv" "/content/gcs"
!gcsfuse --implicit-dirs --rename-dir-limit=100  --max-conns-per-host=100 "outputs_applied_cv_us_central_1" "/content/efficientnet"

mkdir: cannot create directory ‘gcs’: File exists
mkdir: cannot create directory ‘efficientnet’: File exists
{"timestamp":{"seconds":1713885093,"nanos":756674379},"severity":"INFO","message":"Start gcsfuse/2.0.1 (Go version go1.22.1) for app \"\" using mount point: /content/gcs\n"}
{"timestamp":{"seconds":1713885093,"nanos":766523261},"severity":"INFO","message":"GCSFuse mount command flags: {\"AppName\":\"\",\"Foreground\":false,\"ConfigFile\":\"\",\"MountOptions\":{},\"DirMode\":493,\"FileMode\":420,\"Uid\":-1,\"Gid\":-1,\"ImplicitDirs\":true,\"OnlyDir\":\"\",\"RenameDirLimit\":100,\"CustomEndpoint\":null,\"BillingProject\":\"\",\"KeyFile\":\"\",\"TokenUrl\":\"\",\"ReuseTokenFromUrl\":true,\"EgressBandwidthLimitBytesPerSecond\":-1,\"OpRateLimitHz\":-1,\"SequentialReadSizeMb\":200,\"MaxRetrySleep\":30000000000,\"StatCacheCapacity\":20460,\"StatCacheTTL\":60000000000,\"TypeCacheTTL\":60000000000,\"HttpClientTimeout\":0,\"MaxRetryDuration\":-1000000000,\"RetryMultiplier\":2,\"LocalFileC

In [ ]:
! pip install albumentations
! pip install sktime[all_extras]
! pip install sktime[mlflow]
! pip install xgboost
! pip install torchvision
! pip install timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.7/744.7 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 14.6 MB/s eta 0:00:00
     ━━━

# Preliminaries

In [ ]:
import albumentations as A
import gc
import matplotlib.pyplot as plt
import math
# import multiprocessing
import numpy as np
import os
import pandas as pd
import pywt
import random
import time
import torch
import torch.nn as nn
import random
import timm
import librosa
import joblib
import warnings
import copy
import csv
import tqdm
from sktime.utils import mlflow_sktime
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import accuracy_score
import pickle
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F

from albumentations.pytorch import ToTensorV2
from glob import glob
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from typing import Dict, List
from xgboost import XGBClassifier


In [ ]:
device = torch.device("cuda:0")

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(0)

class paths:
    MODEL_WEIGHTS = "/content/efficientnet/outputs/tf_efficientnet_b0_epoch_3_avg_train_loss_0.32836671578192084.pth"
    OUTPUT_DIR = "/content/working/"
    TEST_CSV = "/content/gcs/hms-harmful-brain-activity-classification/test.csv"
    TEST_EEGS= "/content/gcs/hms-harmful-brain-activity-classification/test_eegs/"
    TRAIN_CSV = "/content/gcs/hms-harmful-brain-activity-classification/cleaned_train.csv"
    TRAIN_EEGS= "/content/gcs/hms-harmful-brain-activity-classification/train_eegs/"
    TRAIN_SPECTROGRAMS = "/content/gcs/hms-harmful-brain-activity-classification/train_spectrograms/"
    FEATURE_FOLDER = "/content/gcs/hms-harmful-brain-activity-classification/"
    COMBINED_FEATURES = "/content/gcs/hms-harmful-brain-activity-classification/combined_sorted_features.csv"
    FUSION_DATA = "/content/gcs/hms-harmful-brain-activity-classification/cleaned_train.csv"
    FINAL_FEATURE_FOLDER = "/content/gcs/features/"
    ROCKET_DIR = "/content/gcs/models/rocket/"
    XG_MODEL = "/content/gcs/models/xgboost_model.pkl"


class config:
    BATCH_SIZE = 16
    MODEL = "tf_efficientnet_b0"
    NUM_WORKERS = 2
    PRINT_FREQ = 20
    SEED = 20
    VISUALIZE = False,
    N_EPOCHS = 15
    LR = 1e-3
    LABEL_COLS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
    # not including 'expert_consensus', here


USE_WAVELET = None

NAMES = ['LL','LP','RP','RR']

FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]

okay so efficient net is using the test.csv file and the test_eegs folder which means it uses eeg id patient id and spectrogram id
which is also available in cleaned train.csv

rocket is using cleaned train.csv (just the eeg_id and expert consensus) and train eegs,

xgboost is using the new features.csv 0 and 1

In [ ]:

def spectrogram_from_eeg(parquet_path, display=False):
    # LOAD MIDDLE 50 SECONDS OF EEG SERIES
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg)-10_000)//2
    eeg = eeg.iloc[middle:middle+10_000]

    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((128,256,4),dtype='float32')

    if display:
        plt.figure(figsize=(10,7))
    signals = []
    for k in range(4):
        COLS = FEATS[k]

        for kk in range(4):

            # COMPUTE PAIR DIFFERENCES
            x = eeg[COLS[kk]].values - eeg[COLS[kk+1]].values

            # FILL NANS
            m = np.nanmean(x)
            if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
            else: x[:] = 0

            # DENOISE
            if USE_WAVELET:
                x = denoise(x, wavelet=USE_WAVELET)
            signals.append(x)

            # RAW SPECTROGRAM
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//256,
                  n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)

            # LOG TRANSFORM
            width = (mel_spec.shape[1]//32)*32
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]

            # STANDARDIZE TO -1 TO 1
            mel_spec_db = (mel_spec_db+40)/40
            img[:,:,k] += mel_spec_db

        # AVERAGE THE 4 MONTAGE DIFFERENCES
        img[:,:,k] /= 4.0

        if display:
            plt.subplot(2,2,k+1)
            plt.imshow(img[:,:,k],aspect='auto',origin='lower')
            plt.title(f'EEG {eeg_id} - Spectrogram {NAMES[k]}')

    if display:
        plt.show()
        plt.figure(figsize=(10,5))
        offset = 0
        for k in range(4):
            if k>0: offset -= signals[3-k].min()
            plt.plot(range(10_000),signals[k]+offset,label=NAMES[3-k])
            offset += signals[3-k].max()
        plt.legend()
        plt.title(f'EEG {eeg_id} Signals')
        plt.show()
        print(); print('#'*25); print()

    return img


## XG data

In [ ]:
xg_new_features_base_path = 'gcs/hms-harmful-brain-activity-classification/new_features'
xg_paths = [xg_new_features_base_path + '0.csv', xg_new_features_base_path + '1.csv']
# xg_data = pd.concat([pd.read_csv(path) for path in xg_paths], ignore_index=True)
big_xg_data = pd.read_csv(paths.COMBINED_FEATURES)

counts_per_eeg_id = big_xg_data.groupby('eeg_id').size()

# Calculate the average number of entries per 'eeg_id'
average_entries_per_eeg_id = counts_per_eeg_id.mean()
print(f'average_entries_per_eeg_id {average_entries_per_eeg_id}')

average_entries_per_eeg_id 1.080529411764706


In [ ]:
# xg_unique = big_xg_data.drop_duplicates(subset='eeg_id', keep='first')
# xg_unique.to_csv(paths.COMBINED_FEATURES, index=False)

In [ ]:
xg_data

,Unnamed: 0,eeg_id,LL__variance_larger_than_standard_deviation,LL__has_duplicate_max,LL__has_duplicate_min,LL__has_duplicate,LL__sum_values,LL__abs_energy,LL__mean_abs_change,LL__mean_change,...,RR__fourier_entropy__bins_5,RR__fourier_entropy__bins_10,RR__fourier_entropy__bins_100,RR__permutation_entropy__dimension_3__tau_1,RR__permutation_entropy__dimension_4__tau_1,RR__permutation_entropy__dimension_5__tau_1,RR__permutation_entropy__dimension_6__tau_1,RR__permutation_entropy__dimension_7__tau_1,RR__query_similarity_count__query_None__threshold_0.0,RR__mean_n_absolute_max__number_of_maxima_7
0,0,2022637570,1.0,0.0,0.0,1.0,-235404.787612,2.818751e+07,1.261950,0.006808,...,0.090729,0.136002,0.290926,1.048271,1.436023,1.853268,2.296737,2.763488,NaN,124.670077
1,1,1376681989,1.0,0.0,0.0,1.0,-128191.644260,1.939504e+08,2.068613,-0.003936,...,0.090729,0.136002,0.414868,1.163689,1.692133,2.267913,2.866488,3.465324,NaN,549.654811
2,2,4183293965,1.0,0.0,0.0,1.0,-266079.324068,3.250886e+07,1.712453,0.000145,...,0.090729,0.170467,0.271451,0.774179,0.857372,0.943669,1.031855,1.122134,NaN,2389.748431
3,3,3758850061,1.0,0.0,0.0,1.0,247886.286253,2.236191e+07,1.687947,0.005409,...,0.090729,0.170467,0.395585,0.996659,1.326278,1.678112,2.050456,2.437695,NaN,146.017410
4,4,80871439,1.0,0.0,0.0,1.0,309935.319810,1.608429e+07,0.080873,0.000011,...,0.045395,0.045395,0.136002,0.711625,0.729608,0.747553,0.765722,0.783888,NaN,18.630586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,495,1862928236,1.0,0.0,0.0,0.0,238614.000267,3.799696e+07,2.985318,-0.016036,...,0.125256,0.181214,0.395585,1.114494,1.581580,2.088572,2.628281,3.187571,NaN,134.935569
996,496,3562276716,1.0,0.0,0.0,1.0,-107337.730956,1.310622e+07,1.309241,-0.000896,...,0.136002,0.181214,0.395585,1.122775,1.593995,2.112905,2.660052,3.217764,NaN,146.339602
997,497,4172679021,1.0,0.0,0.0,1.0,-233324.242384,7.488158e+06,0.372072,0.000028,...,0.136002,0.181214,0.271451,0.916202,1.153716,1.404204,1.669672,1.948407,NaN,65.233960
998,498,3139700594,1.0,0.0,0.0,1.0,280144.244961,8.974967e+06,0.830898,-0.000073,...,0.215617,0.325140,1.089700,0.945797,1.215291,1.501093,1.806616,2.125686,NaN,56.633336


In [ ]:
big_xg_data

## rocket data

In [ ]:
train_file = "gcs/hms-harmful-brain-activity-classification/cleaned_train.csv"
train_dir = "gcs/hms-harmful-brain-activity-classification/train_eegs"
rocket_df = df = pd.read_csv(train_file)

In [ ]:
rocket_df

,Unnamed: 0,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,concatenated_scores,is_center
0,40927,568657,0,0.0,789577333,0,0.0,1825637311,20654,Other,0,0,3,0,2,7,3027,False
1,40928,568657,1,6.0,789577333,1,6.0,3640441665,20654,Other,0,0,3,0,2,7,3027,True
2,40929,568657,2,12.0,789577333,2,12.0,1364530340,20654,Other,0,0,3,0,2,7,3027,False
3,40930,568657,3,16.0,789577333,3,16.0,1874525225,20654,Other,0,0,3,0,2,7,3027,False
4,77116,582999,0,0.0,1552638400,0,0.0,1722186807,20230,LPD,0,12,0,1,0,1,120101,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,33556,4294858825,1,2.0,657299228,1,2.0,2581506921,4312,Other,0,0,0,0,1,14,114,False
106796,33557,4294858825,2,6.0,657299228,2,6.0,3251917981,4312,Other,0,0,0,0,1,14,114,True
106797,33558,4294858825,3,8.0,657299228,3,8.0,3646714202,4312,Other,0,0,0,0,1,14,114,False
106798,33559,4294858825,4,12.0,657299228,4,12.0,3219273381,4312,Other,0,0,0,0,1,14,114,False


In [ ]:
# checking data structure:

# Group by 'eeg_id' and calculate the number of unique values for each specified column
unique_values = rocket_df.groupby('eeg_id').agg({'expert_consensus': 'nunique', 'seizure_vote': 'nunique', 'lpd_vote': 'nunique','gpd_vote': 'nunique','lrda_vote': 'nunique','grda_vote': 'nunique','other_vote': 'nunique'})

# Check which 'eeg_id's have more than one unique value in any of the columns
result = unique_values[unique_values > 1].dropna(how='all').fillna(1).mean()

print(result)
filtered_entries = rocket_df[rocket_df['eeg_id'] == 21379701]

counts_per_eeg_id = rocket_df.groupby('eeg_id').size()

# Calculate the average number of entries per 'eeg_id'
average_entries_per_eeg_id = counts_per_eeg_id.mean()
print(f'average_entries_per_eeg_id {average_entries_per_eeg_id}')

expert_consensus    1.511345
seizure_vote        1.722745
lpd_vote            1.703376
gpd_vote            1.326508
lrda_vote           1.443276
grda_vote           1.257333
other_vote          1.863863
dtype: float64
average_entries_per_eeg_id 6.249634267657558


## Joined xg and rocket data

In [ ]:
!du -sh /content/gcs/hms-harmful-brain-activity-classification/combined_sorted_features.csv
!du -sh /content/gcs/hms-harmful-brain-activity-classification/cleaned_train.csv

925M	/content/gcs/hms-harmful-brain-activity-classification/combined_sorted_features.csv
9.2M	/content/gcs/hms-harmful-brain-activity-classification/cleaned_train.csv


In [ ]:
rocket_xg_data = pd.merge(rocket_df, big_xg_data, on='eeg_id', how='left')

In [ ]:
rocket_xg_data

,Unnamed: 0_x,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,...,RR__fourier_entropy__bins_5,RR__fourier_entropy__bins_10,RR__fourier_entropy__bins_100,RR__permutation_entropy__dimension_3__tau_1,RR__permutation_entropy__dimension_4__tau_1,RR__permutation_entropy__dimension_5__tau_1,RR__permutation_entropy__dimension_6__tau_1,RR__permutation_entropy__dimension_7__tau_1,RR__query_similarity_count__query_None__threshold_0.0,RR__mean_n_absolute_max__number_of_maxima_7
0,40927,568657,0,0.0,789577333,0,0.0,1825637311,20654,Other,...,0.159721,0.226363,0.609856,0.967813,1.258921,1.572947,1.909669,2.257855,NaN,595.683271
1,40928,568657,1,6.0,789577333,1,6.0,3640441665,20654,Other,...,0.159721,0.226363,0.609856,0.967813,1.258921,1.572947,1.909669,2.257855,NaN,595.683271
2,40929,568657,2,12.0,789577333,2,12.0,1364530340,20654,Other,...,0.159721,0.226363,0.609856,0.967813,1.258921,1.572947,1.909669,2.257855,NaN,595.683271
3,40930,568657,3,16.0,789577333,3,16.0,1874525225,20654,Other,...,0.159721,0.226363,0.609856,0.967813,1.258921,1.572947,1.909669,2.257855,NaN,595.683271
4,77116,582999,0,0.0,1552638400,0,0.0,1722186807,20230,LPD,...,0.090729,0.170467,0.452944,1.153997,1.659673,2.216935,2.811755,3.431510,NaN,230.746789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,33556,4294858825,1,2.0,657299228,1,2.0,2581506921,4312,Other,...,0.090729,0.136002,0.350689,0.960975,1.247671,1.551857,1.864219,2.182974,NaN,1377.127476
106796,33557,4294858825,2,6.0,657299228,2,6.0,3251917981,4312,Other,...,0.090729,0.136002,0.350689,0.960975,1.247671,1.551857,1.864219,2.182974,NaN,1377.127476
106797,33558,4294858825,3,8.0,657299228,3,8.0,3646714202,4312,Other,...,0.090729,0.136002,0.350689,0.960975,1.247671,1.551857,1.864219,2.182974,NaN,1377.127476
106798,33559,4294858825,4,12.0,657299228,4,12.0,3219273381,4312,Other,...,0.090729,0.136002,0.350689,0.960975,1.247671,1.551857,1.864219,2.182974,NaN,1377.127476


In [ ]:
rocket_xg_data.to_csv(paths.FUSION_DATA, index = False)

KeyboardInterrupt: 

In [ ]:
column_slice = rocket_xg_data.columns[11:17]

# Convert to list if you need a list format
column_list = column_slice.tolist()

In [ ]:
%%time

csv_files = [os.path.join(paths.FEATURE_FOLDER, file) for file in os.listdir(paths.FEATURE_FOLDER) if file.startswith('new_features')]

# Read and concatenate all CSV files into one DataFrame
df_combined = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

# Sort the DataFrame by 'eeg_id'
df_sorted = df_combined.sort_values(by='eeg_id')
df_sorted.reset_index(drop=True, inplace=True).drop_duplicates(subset='eeg_id', keep='first')
# Save the sorted DataFrame to a new CSV file
df_sorted.to_csv(paths.COMBINED_FEATURES, index=False)


print(f"Combined and sorted CSV saved as {paths.COMBINED_FEATURES}")

Combined and sorted CSV saved as /content/gcs/hms-harmful-brain-activity-classification/combined_sorted_features.csv
CPU times: user 2min 52s, sys: 8.18 s, total: 3min
Wall time: 5min 22s


# Reformatting everything

In [ ]:
df_sorted = pd.read_csv(paths.COMBINED_FEATURES)

In [ ]:
df_sorted.shape

(18369, 3110)

In [ ]:
unique_values = df_sorted.nunique()

In [ ]:
unique_values

Unnamed: 0                                                1127
eeg_id                                                   17000
LP__variance_larger_than_standard_deviation                  2
LP__has_duplicate_max                                        2
LP__has_duplicate_min                                        2
                                                         ...  
RR__permutation_entropy__dimension_5__tau_1              16989
RR__permutation_entropy__dimension_6__tau_1              16989
RR__permutation_entropy__dimension_7__tau_1              16989
RR__query_similarity_count__query_None__threshold_0.0        0
RR__mean_n_absolute_max__number_of_maxima_7              16988
Length: 3110, dtype: int64

In [ ]:
#splitting up the features file into little csvs
os.makedirs(paths.FINAL_FEATURE_FOLDER, exist_ok=True)  # Create the directory if it doesn't exist
current_files = set(os.listdir(paths.FINAL_FEATURE_FOLDER))
print(current_files)
print(len(current_files))
for index, row in df_sorted.iterrows():
    # Create a filename using the 'eeg_id'
    filename = f"{paths.FINAL_FEATURE_FOLDER}{str(row['eeg_id'])[:-2]}.csv"
    leaf = str(row['eeg_id'])[:-2]+ '.csv'
    if leaf not in current_files:
      print(',', end = '')
      row_df = pd.DataFrame([row])
      row_df.to_csv(filename, index=False)
    if index % 100 ==0:
        print('.', end = '')

{'2951073689.csv', '957335106.csv', '3446341168.csv', '513666049.csv', '2773247107.csv', '535040610.csv', '82579137.csv', '2366779217.csv', '3514584394.csv', '1244497090.csv', '4087280195.csv', '4213067352.csv', '4245582055.csv', '488622431.csv', '852297681.csv', '1306437510.csv', '1005173045.csv', '1030673804.csv', '859272981.csv', '3379592995.csv', '1879265956.csv', '3826775203.csv', '928882623.csv', '2735897424.csv', '3427295656.csv', '3205748274.csv', '3269046909.csv', '234800101.csv', '505175091.csv', '2915251882.csv', '511079620.csv', '1664133543.csv', '3291882608.csv', '3347948514.csv', '1030375965.csv', '3847510588.csv', '252613483.csv', '401608387.csv', '1235495668.csv', '1372992608.csv', '977185853.csv', '2484812679.csv', '3403081220.csv', '3040584676.csv', '4279600028.csv', '481310657.csv', '3810897651.csv', '653687449.csv', '1560794356.csv', '2258919454.csv', '1161342456.csv', '1397212314.csv', '4071447681.csv', '772806493.csv', '1635866473.csv', '24536592.csv', '138899687.

In [ ]:
current_files = set(os.listdir(paths.FINAL_FEATURE_FOLDER))
print(current_files)
print(len(current_files))

{'2951073689.csv', '957335106.csv', '3446341168.csv', '513666049.csv', '2773247107.csv', '535040610.csv', '82579137.csv', '2366779217.csv', '3514584394.csv', '1244497090.csv', '4087280195.csv', '4213067352.csv', '4245582055.csv', '488622431.csv', '852297681.csv', '1306437510.csv', '1005173045.csv', '1030673804.csv', '859272981.csv', '3379592995.csv', '1879265956.csv', '3826775203.csv', '928882623.csv', '2735897424.csv', '3427295656.csv', '3205748274.csv', '3269046909.csv', '234800101.csv', '505175091.csv', '2915251882.csv', '511079620.csv', '1664133543.csv', '3291882608.csv', '3347948514.csv', '1030375965.csv', '3847510588.csv', '252613483.csv', '401608387.csv', '1235495668.csv', '1372992608.csv', '977185853.csv', '2484812679.csv', '3403081220.csv', '3040584676.csv', '4279600028.csv', '481310657.csv', '3810897651.csv', '653687449.csv', '1560794356.csv', '2258919454.csv', '1161342456.csv', '1397212314.csv', '4071447681.csv', '772806493.csv', '1635866473.csv', '24536592.csv', '138899687.

okay so we will be using the rocket cleaned train df.

from that dataset we get the eeg_id and spectrogram_id which then allows us to map to the feature file in a features folder and a parquet file in train_eegs

So, I am running a script to take the clean big_xg_data file and split each row into its own csv.



filter on is center in cleanedd_train .csv

all of this has been done, so ignore for nwo

# Loading

In [ ]:
fusion_data = pd.read_csv(paths.FUSION_DATA)

In [ ]:
print(fusion_data.nunique())

Unnamed: 0                          21490
eeg_id                              17089
eeg_sub_id                            309
eeg_label_offset_seconds              601
spectrogram_id                      11138
spectrogram_sub_id                    631
spectrogram_label_offset_seconds     2470
label_id                            21490
patient_id                           1950
expert_consensus                        6
seizure_vote                           18
lpd_vote                               19
gpd_vote                               17
lrda_vote                              16
grda_vote                              16
other_vote                             26
concatenated_scores                  3140
is_center                               1
dtype: int64


In [ ]:
class FusionDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        config,
        rocket_model,
        xgboost_model,
    ):
        self.df = df
        self.config = config
        self.rocket_model = rocket_model
        self.xgboost_model = xgboost_model

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        try:
          en_X, y = self.__EN_data_generation(row)
          rocket_in = self.get_rocket_output(row)
          feature_row = pd.read_csv(paths.FINAL_FEATURE_FOLDER + str(row.eeg_id) + '.csv').iloc[:, 2:]
        except FileNotFoundError as e:
          return None

        xg_in = self.get_xgboost_output(feature_row)

        return (torch.tensor(en_X, dtype=torch.float32),
                rocket_in,
                xg_in,
                torch.tensor(y, dtype=torch.float32))

    def get_rocket_output(self,row):
        pq = pd.read_parquet(f"{paths.TRAIN_EEGS}{row.eeg_id}.parquet")
        middle = (len(pq)-2_000)//2
        pq = pq.iloc[middle:middle+2_000:2]
        #just doing this so the formatting works well
        pq = [pq.reset_index()]

        batch_size = 1
        x_batch = pd.concat(pq,keys=list(range(batch_size)),axis=0).reset_index(level=1)
        x_batch['instances'] = x_batch.index
        x_batch = x_batch.rename(columns={"level_1": "timepoints"})
        x_batch = x_batch.set_index(['instances', 'timepoints'])
        x_batch = x_batch.fillna(0)
        rocket_predictions = self.rocket_model.predict(x_batch)
        rocket_predictions = torch.from_numpy(rocket_predictions.to_numpy())
        return rocket_predictions  #, self.df.iloc[ind]['expert_consensus']

    def get_xgboost_output(self,feature_row):
        x = feature_row.values.reshape(1, -1)
        xg_probs = self.xgboost_model.predict_proba(x)[0]
        return torch.tensor(xg_probs, dtype=torch.float32)

    def __EN_data_generation(self, row):
        """
        Generates usable data from parquets
        """
        X = np.zeros((128, 256, 8), dtype='float32')
        y = np.zeros(6, dtype='float32')
        img = np.ones((128,256), dtype='float32')
        # row = self.df.iloc[index]
        # if self.mode=='test':
        #     r = 0
        # else:
        #     r = int((row['min'] + row['max']) // 4)
        r = 0
        spectrogram_file_path = paths.TRAIN_SPECTROGRAMS + str(row.spectrogram_id) + ".parquet"
        spectrogram = pd.read_parquet(spectrogram_file_path).iloc[:,1:].values
        eeg_file_path = paths.TRAIN_EEGS + str(row.eeg_id) + ".parquet"
        eeg = spectrogram_from_eeg(eeg_file_path)

        for region in range(4):
            img = spectrogram[r:r+300, region*100:(region+1)*100].T

            # Log transform spectrogram
            img = np.clip(img, np.exp(-4), np.exp(8))
            img = np.log(img)

            # Standarize per image
            ep = 1e-6
            mu = np.nanmean(img.flatten())
            std = np.nanstd(img.flatten())
            img = (img-mu)/(std+ep)
            img = np.nan_to_num(img, nan=0.0)
            X[14:-14, :, region] = img[:, 22:-22] / 2.0

            # all_eegs[int(eeg_id)] = eeg_spectrogram
            img = eeg
            X[:, :, 4:] = img

            # if self.mode != 'test':
        y = row[config.LABEL_COLS].values.astype(np.float32)
        # print("\n En \n")
        # print(X,y)
        return X, y


def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    X = torch.stack([item[0] for item in batch])
    rocket = torch.stack([item[1] for item in batch])
    xg = torch.stack([item[2] for item in batch])
    y = torch.stack([item[3] for item in batch])

    return {'X': X, 'rocket': rocket, 'xg': xg, 'label': y }


In [ ]:
class EfficientNet(nn.Module):
    def __init__(self, config, num_classes: int = 6):
        super(EfficientNet, self).__init__()
        self.USE_KAGGLE_SPECTROGRAMS = True
        self.USE_EEG_SPECTROGRAMS = True
        self.model = timm.create_model(
            config.MODEL,
            pretrained=False
        )
        self.features = nn.Sequential(*list(self.model.children())[:-2])
        self.custom_layers = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(self.model.num_features, num_classes)
        )

        self.adapter = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten())

        self.featurizer = True

    def __reshape_input(self, x):
        """
        Reshapes input (128, 256, 8) -> (512, 512, 3) monotone image.
        """
        # === Get spectrograms ===
        spectrograms = [x[:, :, :, i:i+1] for i in range(4)]
        spectrograms = torch.cat(spectrograms, dim=1)

        # === Get EEG spectrograms ===
        eegs = [x[:, :, :, i:i+1] for i in range(4,8)]
        eegs = torch.cat(eegs, dim=1)

        # === Reshape (512,512,3) ===
        if self.USE_KAGGLE_SPECTROGRAMS & self.USE_EEG_SPECTROGRAMS:
            x = torch.cat([spectrograms, eegs], dim=2)
        elif self.USE_EEG_SPECTROGRAMS:
            x = eegs
        else:
            x = spectrograms

        x = torch.cat([x,x,x], dim=3)
        x = x.permute(0, 3, 1, 2)
        return x

    def forward(self, x):
        x = self.__reshape_input(x)
        x = self.features(x)
        if self.featurizer:
            x = self.adapter(x)
        else:
            x = self.custom_layers(x)
        return x


class FusionModel(nn.Module):
    def __init__(self, EN_Model, freeze = False):
        super(FusionModel, self).__init__()
        self.EfficientNet = EN_model
        self.EN_out = 1280
        self.encoder_out = 24
        self.hidden = 10
        self.num_classes = 6
        if not freeze:
            for param in self.EfficientNet.parameters():
                param.requires_grad = True

        self.encoder = nn.Linear(self.EN_out, self.encoder_out)
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder_out + 6 + 6, self.hidden),
            nn.ReLU(),
            nn.Linear(self.hidden, self.num_classes)
        )

    def forward(self, X, rocket, xg, label=None):
        EN_features = self.EfficientNet(X)
        EN_features = F.relu(self.encoder(EN_features))
        # for tens  in [EN_features, rocket, xg]:
          # print(tens.size())
        try:
            combined_features = torch.cat((EN_features, rocket.squeeze(), xg), dim=1)
        except:
            print(EN_features, rocket.squeeze(), xg)
            raise
        out = self.classifier(combined_features)
        return F.softmax(out, dim = 0)



# Training

In [ ]:
def train_model(model, criterion, optimizer, dataloaders, num_epochs=5):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    current = time.time()
    with open(f'/content/gcs/fusion/fusion_training_results{str(current)[:2]}.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['epoch', 'phase', 'loss', 'accuracy'])
        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0
                # Iterate over data.
                for _,batch in tqdm(enumerate(dataloaders[phase], 0), unit="batch", total=len(dataloaders[phase])):
                    X = batch['X'].to(device)
                    rocket = batch['rocket'].to(device)
                    xg = batch['xg'].to(device)
                    labels = batch['label'].to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(X, rocket, xg)
                        loss = criterion(outputs, labels/labels.sum())
                        _, preds = torch.max(outputs, 1)
                        _, consensus = torch.max(labels, 1)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * labels.size(0)
                    # print('batch outputs, preds, normed labels')
                    # print(outputs.size())
                    # print(outputs)
                    # print(preds.size())
                    # print(preds)
                    # print(labels.size())
                    # print(labels/labels.sum())
                    running_corrects += torch.sum(preds == consensus.data)


                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]
                print()
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                writer.writerow([epoch, phase, epoch_loss, epoch_acc])

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    print('saving')
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(model.state_dict(), f'/content/gcs/models/fusion_epoch_{epoch}.pth')

            print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    # model.load_state_dict(best_model_wts)
    return model

In [ ]:

fusion_data = pd.read_csv(paths.FUSION_DATA)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    rocket_model = mlflow_sktime.load_model(model_uri=paths.ROCKET_DIR)
    xg_model = joblib.load(paths.XG_MODEL)

fusion_dataset = FusionDataset(fusion_data, config,rocket_model, xg_model)

excess, fusion_dataset = random_split(fusion_dataset, [.95, .05], generator=torch.Generator().manual_seed(0))
train_set, val_set = random_split(fusion_dataset, [.7, .3], generator=torch.Generator().manual_seed(0))
print(len(train_set))
train_loader = DataLoader(train_set,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    num_workers=config.NUM_WORKERS,
    pin_memory=True,
    drop_last=False,
    collate_fn = collate_fn
)
val_loader = DataLoader(val_set,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    num_workers=config.NUM_WORKERS,
    pin_memory=True,
    drop_last=False,
    collate_fn = collate_fn
)

print(len(train_loader))

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train':len(train_loader), 'val': len(val_loader)}

752
47


In [ ]:
%%time
for elem in fusion_dataset[0]:
  print(elem.size())

torch.Size([128, 256, 8])
torch.Size([1, 6])
torch.Size([6])
torch.Size([6])
CPU times: user 2.33 s, sys: 324 ms, total: 2.66 s
Wall time: 3.04 s


In [ ]:
!nvidia-smi

Tue Apr 23 14:07:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0              29W /  70W |   8167MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# print(torch.cuda.memory_allocated())
torch.cuda.empty_cache()

In [ ]:
EN_model = EfficientNet(config)
checkpoint = torch.load(paths.MODEL_WEIGHTS)
EN_model.load_state_dict(checkpoint["model"])
EN_model.to(device)

Fusion_Model = FusionModel(EN_model, freeze=True).to(device)

optimizer = torch.optim.Adam(Fusion_Model.parameters(), lr=config.LR)
# criterion = nn.KLDivLoss()
criterion = nn.CrossEntropyLoss()

In [ ]:
model = train_model(Fusion_Model, criterion, optimizer,dataloaders,num_epochs=config.N_EPOCHS )

Epoch 0/14
----------


  0%|          | 0/47 [00:07<?, ?batch/s]


KeyboardInterrupt: 

In [ ]:
!ls -1 gcs/features | wc -l

17000


# Graveyard

In [ ]:
class RocketDataset(Dataset):
    def __init__(self, train_file, train_path):
        self.df = pd.read_csv(train_file)
        self.dir = train_path
        self.len = len(self.df)
    def __len__(self):
        return self.len
    def __getitem__(self, ind):
        #GETS THE EEG ID AND GOES TO train_eegs, passes in expert consensus label
        eeg_id = self.df.iloc[ind]['eeg_id']
        pq = pd.read_parquet(f"{self.dir}/{eeg_id}.parquet")
        middle = (len(pq)-2_000)//2
        pq = pq.iloc[middle:middle+2_000:2]
        pq = pq.reset_index()

        return pq, self.df.iloc[ind]['expert_consensus']

def collate_fn(batch):
    # convert list of eegs to multi-index df
    # one-hot encode labels

    x_batch = [item[0] for item in batch]
    y_batch = [item[1] for item in batch]
    batch_size = len(x_batch)

    x_batch = pd.concat(x_batch,keys=list(range(batch_size)),axis=0).reset_index(level=1)
    x_batch['instances'] = x_batch.index
    x_batch = x_batch.rename(columns={"level_1": "timepoints"})
    x_batch = x_batch.set_index(['instances', 'timepoints'])
    x_batch = x_batch.fillna(0)

    y_batch = pd.get_dummies(y_batch, columns = ['GPD','GRDA','LPD','LRDA','Other','Seizure']).astype("int32")
    y_batch = y_batch.reset_index(drop=True)

    return x_batch, y_batch

train_file = "gcs/hms-harmful-brain-activity-classification/cleaned_train.csv"
train_dir = "gcs/hms-harmful-brain-activity-classification/train_eegs"
rd = RocketDataset(train_file, train_dir)
train_set, val_set = random_split(rd, [.7, .3], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

class EEGDataset(Dataset):
    def __init__(self, paths, model, label_encoder):
        # load and concat csv
        self.data = pd.concat([pd.read_csv(path) for path in paths], ignore_index=True)
        self.model = model
        self.label_encoder = label_encoder
        self.features = self.data.iloc[:, 2:]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.features.iloc[idx]
        x = x.values.reshape(1, -1)
        # prob distribution
        probs = self.model.predict_proba(x)[0]
        return probs

def collate_fn(batch):
    # create batch
    logits = np.stack(batch)
    return torch.tensor(logits, dtype=torch.float32)

xg_new_features_base_path = 'gcs/hms-harmful-brain-activity-classification/new_features'
xg_paths = [xg_new_features_base_path + '0.csv', new_features_base_path + '1.csv']
xgboost_dataset = EEGDataset(xg_paths=paths, model=model, label_encoder=label_encoder)
xgboost_data_loader = DataLoader(xgboost_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

for logits in data_loader:
    print(logits)
    break



NameError: name 'Dataset' is not defined

,Unnamed: 0,eeg_id,LP__variance_larger_than_standard_deviation,LP__has_duplicate_max,LP__has_duplicate_min,LP__has_duplicate,LP__sum_values,LP__abs_energy,LP__mean_abs_change,LP__mean_change,...,RR__fourier_entropy__bins_5,RR__fourier_entropy__bins_10,RR__fourier_entropy__bins_100,RR__permutation_entropy__dimension_3__tau_1,RR__permutation_entropy__dimension_4__tau_1,RR__permutation_entropy__dimension_5__tau_1,RR__permutation_entropy__dimension_6__tau_1,RR__permutation_entropy__dimension_7__tau_1,RR__query_similarity_count__query_None__threshold_0.0,RR__mean_n_absolute_max__number_of_maxima_7
3267,140,568657,1.0,0.0,0.0,0.0,-391986.353551,1.031208e+08,6.005668,5.587921e-03,...,0.159721,0.226363,0.609856,0.967813,1.258921,1.572947,1.909669,2.257855,NaN,595.683271
11326,457,582999,1.0,0.0,0.0,1.0,-77855.445910,1.072192e+08,1.395476,-3.141775e-04,...,0.090729,0.170467,0.452944,1.153997,1.659673,2.216935,2.811755,3.431510,NaN,230.746789
8166,297,642382,1.0,0.0,0.0,0.0,167115.288372,8.834139e+06,0.908034,-7.986066e-05,...,0.136002,0.245901,0.563773,1.036312,1.410640,1.802202,2.215373,2.645792,NaN,98.728147
14409,40,751790,1.0,0.0,0.0,1.0,-559607.981718,4.080118e+07,1.848305,-2.029073e-04,...,0.215617,0.305728,0.540631,0.920234,1.161361,1.416179,1.683945,1.963932,NaN,193.897661
10891,22,778705,1.0,0.0,0.0,1.0,330348.806905,5.345533e+07,4.008832,-1.314548e-03,...,0.200814,0.294982,0.574520,0.995391,1.318156,1.660623,2.021330,2.401502,NaN,712.750105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10923,54,4293354003,1.0,0.0,0.0,1.0,-95024.909089,1.830831e+07,0.974050,-6.814990e-05,...,0.090729,0.136002,0.305728,0.920250,1.160709,1.416321,1.683730,1.961698,NaN,118.569250
9582,213,4293843368,1.0,0.0,0.0,0.0,66162.234715,4.258600e+07,1.290217,1.403972e-04,...,0.090729,0.170467,0.271451,0.917406,1.151398,1.397892,1.660107,1.934919,NaN,359.148603
3219,92,4294455489,1.0,0.0,0.0,0.0,-290224.452043,1.558077e+09,14.237478,-1.928403e-09,...,0.090729,0.090729,0.271451,0.883903,1.073263,1.224572,1.360286,1.480058,NaN,1679.861328
9408,39,4294858825,1.0,0.0,0.0,1.0,-929134.581197,1.850267e+08,0.522741,5.108982e-02,...,0.090729,0.136002,0.350689,0.960975,1.247671,1.551857,1.864219,2.182974,NaN,1377.127476


In [ ]:

#xgboost
xg_model_path = 'gcs/models/xgboost_model.pkl'
xg_label_encoder_path = 'gcs/models/xgboost_label_encoder.pkl'
xg_model = joblib.load(xg_model_path)
xg_label_encoder = joblib.load(xg_label_encoder_path)

#rocket
model_dir = "rocket/"
rocket_model = mlflow_sktime.load_model(model_uri=model_dir)

#efficientnet

EN_model = EfficientNet(config)
checkpoint = torch.load(paths.MODEL_WEIGHTS)
EN_model.load_state_dict(checkpoint["model"])
EN_model.to(device)
# prediction_dict = inference_function(test_loader, model, device)
# predictions.append(prediction_dict["predictions"])
# torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.4.1.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RidgeClassifierCV from version 1.4.1.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at yo

EfficientNet(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=

In [ ]:
from google.colab import drive
drive.mount('/content/drive')